## Importing Libraries

In [8]:
import random

## TicTacToe Class

- **`reset()`**: Resets the board and winner.
- **`print_board()`**: Displays the board.
- **`make_move(square, letter)`**: Places a marker (`X` or `O`) on the board if valid and checks for a winner.
- **`check_winner(square, letter)`**: Checks if the given player has won after their move.
- **`empty_squares()`**: Returns `True` if there are empty spaces on the board.
- **`available_moves()`**: Lists all valid moves.
- **`is_full()`**: Checks if the board is full.
- **`check_draw()`**: Returns `True` if the game is a draw (full board, no winner).

In [10]:
class TicTacToe:
    def __init__(self):
        self.board = [" " for _ in range(9)]
        self.current_winner = None

    def reset(self):
        self.board = [" " for _ in range(9)]
        self.current_winner = None

    def print_board(self):
        print("   0   1   2")
        for i in range(3):
            row = self.board[i * 3:(i + 1) * 3]
            print(f"{i} | {row[0]} | {row[1]} | {row[2]} |")
            if i < 2:
                print("  ---+---+---")

    def make_move(self, square, letter):
        if self.board[square] == " ":
            self.board[square] = letter
            if self.check_winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def check_winner(self, square, letter):
        row_ind = square // 3
        row = self.board[row_ind * 3:(row_ind + 1) * 3]
        if all([s == letter for s in row]):
            return True

        col_ind = square % 3
        col = [self.board[col_ind + i * 3] for i in range(3)]
        if all([s == letter for s in col]):
            return True

        if square % 2 == 0:
            diag1 = [self.board[i] for i in [0, 4, 8]]
            if all([s == letter for s in diag1]):
                return True

            diag2 = [self.board[i] for i in [2, 4, 6]]
            if all([s == letter for s in diag2]):
                return True

        return False

    def empty_squares(self):
        return " " in self.board

    def available_moves(self):
        return [i for i, x in enumerate(self.board) if x == " "]

    def is_full(self):
        return not self.empty_squares()

    def check_draw(self):
        return self.is_full() and self.current_winner is None


## HumanPlayer Class

- **`get_move(game)`**: 
  - Prompts the player to input their move as `row,col` (e.g., `1,2`).
  - Validates the move to ensure it corresponds to an available square on the board.
  - Returns the valid move as a board index.

In [11]:
class HumanPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        valid_square = False
        val = None
        while not valid_square:
            square = input(f"{self.letter}'s turn. Input move as 'row,col' (e.g., 1,2): ")
            try:
                row, col = map(int, square.split(','))
                val = row * 3 + col
                if val not in game.available_moves():
                    raise ValueError
                valid_square = True
            except (ValueError, IndexError):
                print("Invalid input. Please enter row and column as 'row,col' where row and col are between 0 and 2.")
        return val

## RandomComputerPlayer Class

- **`get_move(game)`**: 
  - Randomly selects a move from the available squares on the board.
  - Uses `random.choice()` to ensure the move is valid.

In [12]:
class RandomComputerPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        return random.choice(game.available_moves())


## Minimax Algorithm
- **Goal**: Simulate all possible game outcomes and select the best move for the computer.
- **Parameters**:
  - `state`: Current game state.
  - `depth`: Depth of recursion.
  - `maximizing_player`: True for the computer's turn; False for the opponent's turn.

- **Base Cases**:
  - Return `1` if the computer wins, `-1` if the opponent wins, or `0` for a draw.

- **Logic**:
  - **Maximizing Player**:
    - Try all moves, simulate outcomes, and return the maximum evaluation score.
  - **Minimizing Player**:
    - Simulate opponent's moves and return the minimum evaluation score.

- **Reference**: https://github.com/Cledersonbc/tic-tac-toe-minimax


## `get_move(game)`
- Iterates over all valid moves.
- Uses `minimax` to calculate the score of each move.
- Tracks the move with the highest score and returns it.


## Key Features
1. **Exploit a Win**: Takes winning moves immediately.
2. **Prevent a Loss**: Blocks the opponent's winning moves.
3. **Strategic Play**: Evaluates moves to maximize future winning chances.


## How It Works
1. Simulate each move on the board.
2. Recursively explore all outcomes using `minimax`.
3. Undo moves after evaluation (backtracking).
4. Return the move with the best score.


In [13]:
class UnbeatableComputerPlayer:
    def __init__(self, letter):
        self.letter = letter

    def minimax(self, state, depth, maximizing_player):
        opponent = "O" if self.letter == "X" else "X"
        if state.current_winner == self.letter:
            return 1
        elif state.current_winner == opponent:
            return -1
        elif state.check_draw():
            return 0

        if maximizing_player:
            max_eval = float('-inf')
            for move in state.available_moves():
                state.make_move(move, self.letter)
                eval = self.minimax(state, depth + 1, False)
                state.board[move] = " "
                state.current_winner = None
                max_eval = max(max_eval, eval)
            return max_eval
        else:
            min_eval = float('inf')
            for move in state.available_moves():
                state.make_move(move, opponent)
                eval = self.minimax(state, depth + 1, True)
                state.board[move] = " "
                state.current_winner = None
                min_eval = min(min_eval, eval)
            return min_eval

    def get_move(self, game):
        best_score = float('-inf')
        best_move = None
        for move in game.available_moves():
            game.make_move(move, self.letter)
            score = self.minimax(game, 0, False)
            game.board[move] = " "
            game.current_winner = None
            if score > best_score:
                best_score = score
                best_move = move
        return best_move


## `play_game_human_vs_human` Function

In [14]:
def play_game_human_vs_human():
    game = TicTacToe()
    human1 = HumanPlayer("X")
    human2 = HumanPlayer("O")

    game.print_board()

    while game.empty_squares():
        for player in [human1, human2]:
            move = player.get_move(game)
            if game.make_move(move, player.letter):
                game.print_board()
                if game.current_winner:
                    print(f"{player.letter} wins!")
                    return
                elif game.check_draw():
                    print("It's a draw!")
                    return

In [18]:
play_game_human_vs_human()

   0   1   2
0 |   |   |   |
  ---+---+---
1 |   |   |   |
  ---+---+---
2 |   |   |   |
   0   1   2
0 | X |   |   |
  ---+---+---
1 |   |   |   |
  ---+---+---
2 |   |   |   |
   0   1   2
0 | X | O |   |
  ---+---+---
1 |   |   |   |
  ---+---+---
2 |   |   |   |
   0   1   2
0 | X | O |   |
  ---+---+---
1 | X |   |   |
  ---+---+---
2 |   |   |   |
   0   1   2
0 | X | O |   |
  ---+---+---
1 | X | O |   |
  ---+---+---
2 |   |   |   |
Invalid input. Please enter row and column as 'row,col' where row and col are between 0 and 2.
   0   1   2
0 | X | O | X |
  ---+---+---
1 | X | O |   |
  ---+---+---
2 |   |   |   |
   0   1   2
0 | X | O | X |
  ---+---+---
1 | X | O |   |
  ---+---+---
2 |   | O |   |
O wins!


## `play_game_human_vs_unbeatable_computer` Function

In [16]:
def play_game_human_vs_unbeatable_computer():
    game = TicTacToe()
    human = HumanPlayer("X")
    computer = UnbeatableComputerPlayer("O")

    game.print_board()

    while game.empty_squares():
        for player in [human, computer]:
            move = player.get_move(game)
            if game.make_move(move, player.letter):
                game.print_board()
                if game.current_winner:
                    print(f"{player.letter} wins!")
                    return
                elif game.check_draw():
                    print("It's a draw!")
                    return


## X is the human player and 0 is the unbeatalbe computer

In [17]:
play_game_human_vs_unbeatable_computer()

   0   1   2
0 |   |   |   |
  ---+---+---
1 |   |   |   |
  ---+---+---
2 |   |   |   |
   0   1   2
0 | X |   |   |
  ---+---+---
1 |   |   |   |
  ---+---+---
2 |   |   |   |
   0   1   2
0 | X |   |   |
  ---+---+---
1 |   | O |   |
  ---+---+---
2 |   |   |   |
   0   1   2
0 | X | X |   |
  ---+---+---
1 |   | O |   |
  ---+---+---
2 |   |   |   |
   0   1   2
0 | X | X | O |
  ---+---+---
1 |   | O |   |
  ---+---+---
2 |   |   |   |
Invalid input. Please enter row and column as 'row,col' where row and col are between 0 and 2.
   0   1   2
0 | X | X | O |
  ---+---+---
1 |   | O |   |
  ---+---+---
2 | X |   |   |
   0   1   2
0 | X | X | O |
  ---+---+---
1 | O | O |   |
  ---+---+---
2 | X |   |   |
   0   1   2
0 | X | X | O |
  ---+---+---
1 | O | O | X |
  ---+---+---
2 | X |   |   |
   0   1   2
0 | X | X | O |
  ---+---+---
1 | O | O | X |
  ---+---+---
2 | X | O |   |
   0   1   2
0 | X | X | O |
  ---+---+---
1 | O | O | X |
  ---+---+---
2 | X | O | X |
It's a draw!


An unbeatable computer player this class would create by making use of the minimax method, which goes recursively through all game outcomes by simulating moves and testing win/loss/draw conditions, alternating between maximising the chances for computer and minimising them for an opponent. The method available_moves() returns the space of possible actions and then tests valid moves to find the best. All feedback such as move-making or updating the game states is inherently taken care of by the TicTacToe class, which is meant to manage board interaction and win or draw checks. Likewise, the very tightly connected and acting code in minimax would allow playing perfectly without needing explicit helper methods, though some conceptual things, such as simulating moves or evaluating outcomes, lie buried in the recursiveness.